## Notation for solving the relocating problem in [Braverman et al. 2019]

### Part 1: The basics
#### *Parameters*
$r \in \mathbb{R}_+$: Number of regions 

$N \in \mathbb{R}_+$: Number of robotaxies in the fleet 

$\lambda \in \mathbb{R}^r_+$: A vector for riders' arrival rate (Poisson) across all regions <br>
[we can also use $\lambda := N\cdot\lambda$ to make it dependent on fleet size.] 

$P \in \mathbb{R}^{rxr}_+$: State transition matrix for a rider, allowing $P_{ii}>0$

$\mu \in \mathbb{R}^{rxr}_+$: Reciprocal of mean travel time, $1/\mu_{ij}$ is the mean travel time from region $i$ to region $j$ for a robotaxi <br>
[we can assume travel times from $i$ to $j$ for all robotaxies are i.i.d. exponential r.v. with mean $1/\mu_{ij}$]

#### *To solve*:
$Q \in \mathbb{R}^{rxr}_+$: State transition matrix for relocating a robotaxi (when it is empty)

#### *Other variables*:
$E^{(N)}(t) \in \mathbb{Z}_+^{rxr}, t\geq 0$: Empty-car distribution matrix at flet size $N$ and time step $t$, <br> $E^{(N)}_{ij}(t)$ = \# of empty cars going from $i$ to $j$ or stays in $i$ if $j=i$ <br>
$\bar{E}^{(N)}(t) = E^{(N)}(t)/N$

$F^{(N)}(t) \in \mathbb{Z}_+^{rxr}, t\geq 0$: Occupied-car distribution matrix at flet size $N$ and time step $t$, <br> $F^{(N)}_{ij}(t)$ = \# of occupied cars going from $i$ to $j$ or stays in $i$ if $j=i$ <br>
$\bar{F}^{(N)}(t) = F^{(N)}(t)/N$ <br>
$\bar{E}^{(N)}(\infty), \bar{F}^{(N)}(\infty)$ are the stationary distribution of the $(\bar{E}^{(N)}, \bar{F}^{(N)})$ CTMC. <br>
This CTMC has a single recurrence class, so all regions are reachable from any other region.

$\mathcal{J} = \{(e,f)\in [0,1]^{rxr}\times [0,1]^{rxr}: \sum_{i,j=1}^r (e_{ij}+f_{ij}) = 1\}$ <br>
This is the set of state space (empty-car percentage over all (stay+en route) x occupied-car percentage over all (stay+en route)); the total sum is 1 which is the entire fleet.

$A^{(N)}(\infty) \in [0,1]^r$ where $A_i^{(N)}(\infty) = \mathbb{P}(\bar{E}^{(N)}_{ii}(\infty)>0)$

### Part 2: The optimization problem to solve an optimal static relocating policy

$c_{ij} > 0$: Reward for completing a trip from $i$ to $j$

$q \in \mathbb{R}^{rxr}$: $q_{ij}$'s are decision variables representing the static policy

$(\bar{e}, \bar{f}, \bar{a}) \in \mathcal{J}\times [0,1]^r$ <br>
$\bar{e} = \mathbb{E}[\bar{E}^{(N)}(\infty)], \bar{f} = \mathbb{E}[\bar{F}^{(N)}(\infty)], \bar{a} = A^{(N)}(\infty)$

$\bar{a}_i \lambda_i P_{ij}$: The rate that a ride (car w/ rider) which is from $i$ to $j$ arrives

$\bar{a}_i = \mathbb{P}(\bar{E}^{(N)}_{ii}(\infty)>0)$

### Formulation \# 1

\begin{align}
\max_{q,\bar{e},\bar{f},\bar{a}}\;\sum_{i=1}^r &\sum_{j=1}^r \bar{a}_i\lambda_i P_{ij}c_{ij}\\
\text{subject to } \bar{f}_{ij} &= \frac{1}{\mu_{ij}}\bar{a}_i\lambda_iP_{ij} &\forall i,j=1,\cdots,r\quad \text{Little's Law }L=W\lambda \text{ for occupied cars going from $i$ to $j$}\\
\bar{e}_{ij} &= \frac{1}{\mu_{ij}}q_{ij}\sum_{k=1}^r \mu_{ki}\bar{f}_{ki} &\forall 1 \leq i\neq j=1 \leq r\quad\text{Little's Law }L=W\lambda \text{ for empty cars cars going from $i$ to $j$}\\
\text{RHS } &-> \text{ the rate that empty cars start their trips}\\
\lambda_i \bar{a}_i &= \sum_{k=1,k\neq i}^r \mu_{ki}\bar{e}_{ki} + q_{ii}\sum_{k=1}^r\mu_{ki}\bar{f}_{ki} &\forall i=1,\cdots,r\quad\text{Car flow balance at region $i$}\\
(1- &\bar{a}_i)\bar{e}_{ii} = 0 &\forall i=1,\cdots,r\quad\text{Availability and empty car relation}\\
(\bar{e},&\bar{f}) \in \mathcal{J} &\text{ unit mass constraints }\\
q_{ij}&\geq 0,\quad \sum_{j=1}^r q_{ij} = 1,\quad 0\leq \bar{a}_{i}\leq 1
\end{align}
For car flow balance, LHS is total outflow rate (rider requests) from region $i$ (rider arrival rate x if empty car is available); RHS is total inflow rate (from other regions and ones that stays in $i$)<br>
For availability, $1-\bar{a}_i  = \mathbb{P}(\bar{E}^{(N)}_{ii}(\infty)=0)$, so $1-\bar{a}_i = 0$ *iff* $\bar{e}_{ii}>0$


#### Remark on Formulation \# 1 
From Section 3.2 of **Braverman et al. 2019**:
The fluid-based empty-car routing policy requires 2 important assumptions: <br>
1. The parameters remain constant over time
2. The system has reached equilibrium

Common solution: divide the day into periods of time where parameters are approximately constant w.r.t. each period.

### T-lookahead policy

For time window $[t, t+T]$:

\begin{align}
\max_{q,\bar{e},\bar{f},\bar{a}}\;\frac{1}{T}\int_0^T\bigg(\sum_{i=1}^r &\sum_{j=1}^r \bar{a}_i\lambda_i(t+u)\cdot P_{ij}(t+u)\cdot c_{ij}(t+u) du\bigg)\\
\text{subject to } \bar{f}_{ij} &= \frac{1}{T}\int_0^T\bigg(\frac{1}{\mu_{ij}(t+u)}\bar{a}_i\lambda_i(t+u)\cdot P_{ij}(t+u) du\bigg) 
\quad\forall i,j=1,\cdots,r\quad \text{Little's Law }L=W\lambda \text{ for occupied cars going from $i$ to $j$}\\
\bar{e}_{ij} &= \frac{1}{T}\int_0^T\bigg(\frac{1}{\mu_{ij}(t+u)}q_{ij}\sum_{k=1}^r \mu_{ki}(t+u)\bar{f}_{ki} du\bigg)
\quad\forall 1 \leq i\neq j=1 \leq r\quad\text{Little's Law }L=W\lambda \text{ for empty cars cars going from $i$ to $j$}\\
\text{RHS } &-> \text{ the rate that empty cars start their trips}\\
\lambda_i \bar{a}_i &= \frac{1}{T}\int_0^T\bigg( \sum_{k=1,k\neq i}^r \mu_{ki}(t+u)\bar{e}_{ki} + q_{ii}\sum_{k=1}^r\mu_{ki}(t+u)\bar{f}_{ki}\bigg)
\quad\forall i=1,\cdots,r\quad\text{Car flow balance at region $i$}\\
(1- &\bar{a}_i)\bar{e}_{ii} = 0 \quad\forall i=1,\cdots,r\quad\text{Availability and empty car relation}\\
(\bar{e},&\bar{f}) \in \mathcal{J} \quad\text{ unit mass constraints }\\
q_{ij}&\geq 0,\quad \sum_{j=1}^r q_{ij} = 1,\quad 0\leq \bar{a}_{i}\leq 1
\end{align}